# Wokflow  **Entrega** con zLightGBM

## Inicializacion

In [25]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-11-14 11:56:57 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,869067,46.5,1454668,77.7,1454668,77.7
Vcells,1588933,12.2,2936383694,22402.9,2845722679,21711.2


In [ ]:
plocal <- list()

# 503
plocal$qcanaritos <- 5L
plocal$min_data_in_leaf <- 50L
plocal$learning_rate <- 1.0
plocal$gradient_bound <- 0.01


plocal$APO <- 1
plocal$ksemillerio <- 3


In [ ]:
PARAM <- list()
PARAM$experimento <- "mv_zLGBM_E_001"
PARAM$semilla_primigenia <- 611033

PARAM$train_final$future <- c(202108)

PARAM$train_final$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106
)

# ToDo: Revisar esta decisión
PARAM$train_final$undersampling <- 0.25 # Orsi y Gómez


# Palancas
quitar_prestamos_personales <- TRUE
quitar_variables_rotas <- FALSE
aplicar_correccion_por_quarter <- TRUE
calcular_tendencias <- TRUE
aplicar_data_drifting_ipc <- FALSE
agregar_randomforest <- FALSE
tipo_de_undersampling <- "NAIVE" #"CLIENT-WISE"


In [28]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

## Preprocesamiento

### Generacion de la clase_ternaria

In [29]:
Sys.time()
require( "data.table" )

# leo el dataset
dataset <- fread("~/datasets/competencia_02_crudo.csv.gz" )

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

[1] "2025-11-14 11:57:01 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,870195,46.5,1454668,77.7,1454668,77.7
Vcells,722303243,5510.8,2349106956,17922.3,2845722679,21711.2


[1] "2025-11-14 11:57:24 UTC"

In [30]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,645
201901,BAJA+2,729
201901,CONTINUA,122899
201902,BAJA+1,733
201902,BAJA+2,707
201902,CONTINUA,123961
201903,BAJA+1,708
201903,BAJA+2,751
201903,CONTINUA,124508


### Eliminacion de Features

Completar a gusto LUEGO de realizar un analisis exploratorio de datos.
<br> No necesariamente en esta Segunda Competencia conviele eliminar los mismos campos que en la Primera ...

In [ ]:
# ToDo: Ver si hay otras variables para quitar
if (quitar_prestamos_personales) {
    cat("Quito prestamos personales\n")
    # Salsa Magica para 202106
    dataset[, mprestamos_personales := NULL ]
    dataset[, cprestamos_personales := NULL ]
} else {
cat("No quito prestamos personales\n")
}

No quito prestamos personales

# Eliminación de Variables rotas
Estas son variables que tienen todos los valores en cero en un determinado mes

In [ ]:
if (quitar_variables_rotas) {
    
    cat("Detectando y Quitando variables rotas\n")
    # asumimos que `dataset` ya es un data.table y tiene la columna foto_mes

    # 1) columnas numéricas candidatas (excluimos foto_mes)
    cols_num <- setdiff(names(dataset)[sapply(dataset, is.numeric)], "foto_mes")

    # 2) detectar, por mes, qué columnas están TODAS en 0
    # isTRUE(all(v == 0)) devuelve TRUE solo si no hay NAs y todo es 0; si hay NAs -> FALSE
    flag_wide <- dataset[
    , lapply(.SD, function(v) isTRUE(all(v == 0))), by = .(foto_mes), .SDcols = cols_num
    ]

    # Pasar a largo y quedarnos con las que son TRUE (rotas)
    rotas <- melt(
    flag_wide, id.vars = "foto_mes",
    variable.name = "variable", value.name = "all_zero"
    )[all_zero == TRUE, .(foto_mes, variable)]

    # 3) reemplazar por NA esas variables en esos meses (respetando tipo de columna)
    if (nrow(rotas) > 0) {
    for (k in seq_len(nrow(rotas))) {
        col <- rotas$variable[k]
        mes <- rotas$foto_mes[k]
        na_val <- if (is.integer(dataset[[col]])) NA_integer_ else NA_real_
        set(dataset, i = which(dataset$foto_mes == mes), j = col, value = na_val)
        }
    }

    # `rotas` queda como la lista de variables detectadas por mes
    rotas[]
} else {
    cat("No quito variables rotas\n")
}

# Data Drifting

Se debe corregir el drifting natural que ocurre en loa datos, en particular los datos monetarios que se vieron fuertemente afectados por una alta inflación
<br> Posibles métodos son:
* No hacer absolutamente nada
* Ajuste de valores monetarios por indices del tipo :
   * IPC  Indice de Precios al Consumidor
   * Dolar Oficial
   * Dolar Blue
   * UVA  Unidad de Valor Adquisitivo

a este codigo de Data Drifting lo debera escribir usted

In [ ]:
# Tabla de IPC tomada de: https://www.indec.gob.ar/indec/web/Nivel4-Tema-3-5-31
tabla_ipc <- data.table(
    foto_mes = c(201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 
    201910, 201911, 201912, 202001, 202002, 202003, 202004, 202005, 202006, 
    202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 
    202104, 202105, 202106, 202107, 202108),
    ipc_nucleo = c(3.0, 3.9, 4.6, 3.8, 3.2, 2.7, 2.1, 4.6, 6.4, 3.8, 4.0, 3.7, 2.4, 2.4, 
    3.1, 1.7, 1.6, 2.3, 2.5, 3.0, 2.3, 3.5, 3.9, 4.9, 3.9, 4.1, 4.5, 4.6, 
    3.5, 3.6, 3.1, 3.1)
)

variables_a_indexar <- c("mrentabilidad", "mrentabilidad_annual", "mcomisiones",
"mactivos_margen","mpasivos_margen","mcuenta_corriente","mcaja_ahorro",
"mcaja_ahorro_adicional","mcaja_ahorro_dolares","mcuentas_saldo","mautoservicio",
"mtarjeta_visa_consumo","mtarjeta_master_consumo","mprestamos_prendarios",
"mprestamos_hipotecarios","mplazo_fijo_pesos","minversion1_pesos","minversion2",
"mpayroll","mpayroll2","mcuenta_debitos_automaticos","mttarjeta_visa_debitos_automaticos",
"mttarjeta_master_debitos_automaticos","mpagodeservicios","mpagomiscuentas",
"mcajeros_propios_descuentos","mtarjeta_visa_descuentos","mtarjeta_master_descuentos",
"mcomisiones_mantenimiento","mcomisiones_otras","mforex_buy","mforex_sell",
"mtransferencias_recibidas","mtransferencias_emitidas","mextraccion_autoservicio",
"mcheques_depositados","mcheques_emitidos","mcheques_depositados_rechazados",
"mcheques_emitidos_rechazados","matm","matm_other","Master_mfinanciacion_limite",
"Master_mconsumospesos","Master_mlimitecompra","Master_madelantopesos","Master_mpagado",
"Master_mpagospesos","Master_mconsumototal","Master_mpagominimo",
"Visa_mfinanciacion_limite","Visa_msaldototal","Visa_msaldopesos",
"Visa_mconsumospesos","Visa_mlimitecompra","Visa_madelantopesos",
"Visa_mpagado","Visa_mpagospesos","Visa_mconsumototal","Visa_mpagominimo")

if (!quitar_prestamos_personales) {
    variables_a_indexar <- c(variables_a_indexar, "mprestamos_personales")
}


In [ ]:
if (aplicar_data_drifting_ipc) {
    cat("Aplicando la corrección de Data Drifting por IPC \n")
    # --- Supuestos ---
    # dataset: data.table con columna foto_mes (YYYYMM, numérico/integer)
    # tabla_ipc: data.table con columnas foto_mes (YYYYMM) e IPC (% mensual vs mes anterior)
    # variables_a_indexar: character vector con nombres de columnas numéricas a deflactar
    # objetivo: expresar montos en moneda constante de 201901

    # 1) Definir mes base
    mes_base <- 201901L

    # 2) Asegurar orden y calcular factor acumulado de precios (base = 1 en 201901)
    #    IPC está en % mensual -> factor mensual = 1 + IPC/100
    #    factor_acum en 201901 = 1; luego se compone multiplicando mes a mes
    ipc <- copy(tabla_ipc)
    setorder(ipc, foto_mes)

    ipc[, factor_mes := 1 + ipc_nucleo/100]
    ipc[foto_mes == mes_base, factor_mes := 1]                 # base = 1 (ignoramos su IPC respecto a dic-2018)
    ipc[, factor_acum := cumprod(factor_mes)]

    # 3) Unir factor al dataset
    #    (queda una columna 'factor_acum' en dataset para cada fila/mes)
    dataset[ipc[, .(foto_mes, factor_acum)], on = "foto_mes", factor_acum := i.factor_acum]

    # 4) Crear columnas ajustadas a moneda constante 201901
    #    monto_const = monto_nominal / factor_acum
    #    (si te interesara llevar a moneda de otro mes base, multiplicás por factor_base/ factor_mes)
    sufijo <- "_const201901"

    for (col in variables_a_indexar) {
        if (col %in% names(dataset)) {
            cat("Corrigiendo columna: ", col, "\n")
            dataset[, (col) := get(col) / factor_acum]
        } else {
        cat("⚠️  La columna no existe en dataset: ", col, "\n")
        }
    }

    # 5) limpiar columna auxiliar
    dataset[, factor_acum := NULL]
    cat("✔️  Corrección por IPC aplicada a", length(variables_a_indexar), "columnas.\n")
    

} else {
    cat("Pasando por alto la corrección de Data Drifting por IPC \n")
}

### Feature Engineering Intra-Mes

Crear variables nuevas a partir de las existentes dentro del mismo registro, **sin** ir a buscar información histórica.
<br> El siguiente código es un mínimo ejemplo, agregar nuevos features a gusto

In [32]:
if (aplicar_correccion_por_quarter) {
    cat("Aplicando corrección por quarter\n")
    # el mes 1,2, ..12 , podria servir para detectar estacionalidad
    dataset[, kmes := foto_mes %% 100]

    # creo un ctr_quarter que tenga en cuenta cuando
    # los clientes hace 3 menos meses que estan
    # ya que seria injusto considerar las transacciones medidas en menor tiempo
    dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]
    dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5.0]
    dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2.0]
    dataset[cliente_antiguedad == 3, ctrx_quarter_normalizado := ctrx_quarter * 1.2]

    # variable extraida de una tesis de maestria de Irlanda, se perdió el link
    dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]
} else {
    cat("Salteando la corrección por quarter\n")
}

Sys.time()

Salteando la corrección por quarter

[1] "2025-11-14 11:57:25 UTC"

### Feature Engineering Historico

In [33]:
if( !require("Rcpp")) install.packages("Rcpp", repos = "http://cran.us.r-project.org")
require("Rcpp")

In [34]:
# se calculan para los 6 meses previos el minimo, maximo y
#  tendencia calculada con cuadrados minimos
# la formula de calculo de la tendencia puede verse en
#  https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
# para la maxíma velocidad esta funcion esta escrita en lenguaje C,
# y no en la porqueria de R o Python

cppFunction("NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde )
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) )
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      {
        xsum  += x[h] ;
        ysum  += y[h] ;
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ;
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}")

In [35]:
# calcula la tendencia de las variables cols de los ultimos 6 meses
# la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
# La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas <- function(
    dataset, cols, ventana = 6, tendencia = TRUE,
    minimo = TRUE, maximo = TRUE, promedio = TRUE,
    ratioavg = FALSE, ratiomax = FALSE) {
  gc(verbose= FALSE)
  # Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion <- ventana

  last <- nrow(dataset)

  # creo el vector_desde que indica cada ventana
  # de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids <- dataset[ , numero_de_cliente ]

  vector_desde <- seq(
    -ventana_regresion + 2,
    nrow(dataset) - ventana_regresion + 1
  )

  vector_desde[1:ventana_regresion] <- 1

  for (i in 2:last) {
    if (vector_ids[i - 1] != vector_ids[i]) {
      vector_desde[i] <- i
    }
  }
  for (i in 2:last) {
    if (vector_desde[i] < vector_desde[i - 1]) {
      vector_desde[i] <- vector_desde[i - 1]
    }
  }

  for (campo in cols) {
    nueva_col <- fhistC(dataset[, get(campo)], vector_desde)

    if (tendencia) {
      dataset[, paste0(campo, "_tend", ventana) :=
        nueva_col[(0 * last + 1):(1 * last)]]
    }

    if (minimo) {
      dataset[, paste0(campo, "_min", ventana) :=
        nueva_col[(1 * last + 1):(2 * last)]]
    }

    if (maximo) {
      dataset[, paste0(campo, "_max", ventana) :=
        nueva_col[(2 * last + 1):(3 * last)]]
    }

    if (promedio) {
      dataset[, paste0(campo, "_avg", ventana) :=
        nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratioavg) {
      dataset[, paste0(campo, "_ratioavg", ventana) :=
        get(campo) / nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratiomax) {
      dataset[, paste0(campo, "_ratiomax", ventana) :=
        get(campo) / nueva_col[(2 * last + 1):(3 * last)]]
    }
  }
}

In [ ]:
# Feature Engineering Historico
# Creacion de LAGs: Esto se hace siempre
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
  colnames(dataset),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
))

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
  paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 2
dataset[,
  paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
  dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
  dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}

Sys.time()

[1] "2025-11-14 11:59:11 UTC"

In [37]:
# parametros de Feature Engineering Historico de Tendencias
PARAM$FE_hist$Tendencias$run <- TRUE
PARAM$FE_hist$Tendencias$ventana <- 6
PARAM$FE_hist$Tendencias$tendencia <- TRUE
PARAM$FE_hist$Tendencias$minimo <- FALSE
PARAM$FE_hist$Tendencias$maximo <- FALSE
PARAM$FE_hist$Tendencias$promedio <- FALSE
PARAM$FE_hist$Tendencias$ratioavg <- FALSE
PARAM$FE_hist$Tendencias$ratiomax <- FALSE

In [38]:
# aqui se agregan las tendencias de los ultimos 6 meses
if (calcular_tendencias) {
    cat("Ejecutando cálculo de tendencias\n")
    cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
    setorder(dataset, numero_de_cliente, foto_mes)

    if( PARAM$FE_hist$Tendencias$run) {
        TendenciaYmuchomas(dataset,
        cols = cols_lagueables,
        ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
        tendencia = PARAM$FE_hist$Tendencias$tendencia,
        minimo = PARAM$FE_hist$Tendencias$minimo,
        maximo = PARAM$FE_hist$Tendencias$maximo,
        promedio = PARAM$FE_hist$Tendencias$promedio,
        ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
        ratiomax = PARAM$FE_hist$Tendencias$ratiomax
      )
    }
} else cat("Pasando por alto el cálculo de tendencias\n")
    
ncol(dataset)
Sys.time()

Pasando por alto el cálculo de tendencias

[1] 763

[1] "2025-11-14 11:59:11 UTC"

# Feature Engineering a partir de hojas de RF
Creamos features a partir de la pertenencia a hojas de un RF pre-construido.

In [ ]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

AgregaVarRandomForest <- function() {

  cat( "inicio AgregaVarRandomForest()\n")
  gc(verbose= FALSE)
  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% c( "BAJA+2", "BAJA+1"),
      clase01 := 1L ]

  campos_buenos <- setdiff(
    colnames(dataset),
    c( "clase_ternaria", "clase01")
  )

  dataset[, entrenamiento :=
    as.integer( foto_mes %in% PARAM$FE_rf$train$training )]

  dtrain <- lgb.Dataset(
    data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    free_raw_data = FALSE
  )

  modelo <- lgb.train(
     data = dtrain,
     param = PARAM$FE_rf$lgb_param,
     verbose = -100
  )

  cat( "Fin construccion RandomForest\n" )
  # grabo el modelo, achivo .model
  lgb.save(modelo, file="modelo.model" )

  qarbolitos <- copy(PARAM$FE_rf$lgb_param$num_iterations)

  periodos <- dataset[ , unique( foto_mes ) ]

  for( periodo in  periodos )
  {
    cat( "periodo = ", periodo, "\n" )
    datamatrix <- data.matrix(dataset[ foto_mes== periodo, campos_buenos, with = FALSE])

    cat( "Inicio prediccion\n" )
    prediccion <- predict(
        modelo,
        datamatrix,
        type = "leaf"
    )
    cat( "Fin prediccion\n" )

    for( arbolito in 1:qarbolitos )
    {
       cat( arbolito, " " )
       hojas_arbol <- unique(prediccion[ , arbolito])

       for (pos in 1:length(hojas_arbol)) {
         # el numero de nodo de la hoja, estan salteados
         nodo_id <- hojas_arbol[pos]
         dataset[ foto_mes== periodo, paste0(
            "rf_", sprintf("%03d", arbolito),
             "_", sprintf("%03d", nodo_id)
          ) :=  as.integer( nodo_id == prediccion[ , arbolito]) ]

       }

       rm( hojas_arbol )
    }
    cat( "\n" )

    rm( prediccion )
    rm( datamatrix )
    gc(verbose= FALSE)
  }

  gc(verbose= FALSE)

  # borro clase01 , no debe ensuciar el dataset
  dataset[ , clase01 := NULL ]

}

In [ ]:
# Parametros de Feature Engineering  a partir de hojas de Random Forest

# Estos CUATRO parametros son los que se deben modificar
# ToDo: Encontrar variantes para estos parámetros
PARAM$FE_rf$arbolitos= 20
PARAM$FE_rf$hojas_por_arbol= 16
PARAM$FE_rf$datos_por_hoja= 100
PARAM$FE_rf$mtry_ratio= 0.2

# Estos son quasi fijos
PARAM$FE_rf$train$training <- c( 202101, 202102, 202103)

# Estos TAMBIEN son quasi fijos
PARAM$FE_rf$lgb_param <-list(
    # parametros que se pueden cambiar
    num_iterations = PARAM$FE_rf$arbolitos,
    num_leaves  = PARAM$FE_rf$hojas_por_arbol,
    min_data_in_leaf = PARAM$FE_rf$datos_por_hoja,
    feature_fraction_bynode  = PARAM$FE_rf$mtry_ratio,

    # para que LightGBM emule Random Forest
    boosting = "rf",
    bagging_fraction = ( 1.0 - 1.0/exp(1.0) ),
    bagging_freq = 1.0,
    feature_fraction = 1.0,

    # genericos de LightGBM
    max_bin = 31L,
    objective = "binary",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    force_row_wise = TRUE,
    verbosity = -100,
    max_depth = -1L,
    min_gain_to_split = 0.0,
    min_sum_hessian_in_leaf = 0.001,
    lambda_l1 = 0.0,
    lambda_l2 = 0.0,

    pos_bagging_fraction = 1.0,
    neg_bagging_fraction = 1.0,
    is_unbalance = FALSE,
    scale_pos_weight = 1.0,

    drop_rate = 0.1,
    max_drop = 50,
    skip_drop = 0.5,

    extra_trees = FALSE
  )


In [ ]:
# Feature Engineering agregando variables de Random Forest
#  aqui es donde se hace el trabajo
if (agregar_randomforest) {
    cat("Agregando Variables Random Forest \n")
    AgregaVarRandomForest()
    ncol(dataset)
    colnames(dataset)
} else {
    cat("Pasando por alto el agregado de Variables Random Forest \n")
}
Sys.time()

## Modelado

No hay modelado, no se hace optimizacion de hiperparametros.

## Produccion

Las decisiones que se toman para la construccion del modelo final son:
* Los positvos son  POS={"BAJA+1", "BAJA+2"}, esta es una meticulosa decisión.
* Se entrena en los treinta meses del intervalo [201901, 202104]
* Se realiza undersampling al 5%
* Se utilizan los hiperparámetros optimos encontrados en la Bayesian Optimization
   * Se escala min_data_in_leaf

### Final Training Strategy
Entreno en todos los meses hasta 202106
Hago un undersampling de 0.25 de la clase mayoritaria usando resultados de Gomez y Orsi

In [ ]:
# Moví los parámetros arriba

In [40]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$training]

In [41]:
# canaritos
PARAM$train_final$lgbm$qcanaritos <- plocal$qcanaritos

cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)

if( PARAM$train_final$lgbm$qcanaritos > 0 ) {
  for( i in seq(PARAM$train_final$lgbm$qcanaritos) ){
    dataset_train_final[, paste0("canarito_",i) := runif( filas) ]
  }

  # las columnas canaritos mandatoriamente van al comienzo del dataset
  cols_canaritos <- copy( setdiff( colnames(dataset_train_final), cols0 ) )
  setcolorder( dataset_train_final, c( cols_canaritos, cols0 ) )
}

Sys.time()

[1] "2025-11-14 11:59:18 UTC"

#### Registros cambio las proporciones de POS/NEG

In [ ]:
clientwise_undersampling <- function(dataset_in, undersampling_value, semilla) {

    dataset <- copy(dataset_in)
    # Fijar semilla para reproducibilidad
    set.seed(semilla, kind = "L'Ecuyer-CMRG")

    # Paso 1: Crear tabla con un registro por cliente
    clientes <- dataset[, 
      .(tiene_baja = any(clase_ternaria %in% c("BAJA+1", "BAJA+2"))),
      by = numero_de_cliente
    ]

    # Paso 2: Decidir si se incluye el cliente en el set de entrenamiento
    clientes[, incluir := fifelse(
      tiene_baja,
      TRUE,
      runif(.N) <= undersampling_value
    )]

    # Paso 3: Inicializar la columna 'training' en el dataset original
    dataset[, training := 0L]

    # Paso 4: Marcar como training las filas de clientes seleccionados, solo en meses de entrenamiento
    dataset[
        numero_de_cliente %in% clientes[incluir == TRUE, numero_de_cliente],
        training := 1L
    ]
    return(dataset)
}


contar_filas <- function(dataset_in) {
    cant_clientes <- uniqueN(dataset_in$numero_de_cliente)
    cant_filas <- nrow(dataset_in)
    cat("Clientes únicos en dataset completo:", cant_clientes, "\n")
    cat("Filas totales en dataset completo:", cant_filas, "\n")
    cat("Clientes únicos en dataset de entrenamiento:", uniqueN(dataset_in[training == 1, numero_de_cliente]), "\n")
    cat("Filas totales en dataset de entrenamiento:", nrow(dataset_in[training == 1]), "\n")  
    cat("Clientes únicos en filas filtradas:", uniqueN(dataset_in[training == 0, numero_de_cliente]), "\n")
    cat("Filas totales en filas filtradas:", nrow(dataset_in[training == 0]), "\n")
}


In [ ]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

if (tipo_de_undersampling == "NAIVE") {
    cat("Aplicando NAIVE undersampling\n")
    set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
    dataset_train_final[, azar := runif(nrow(dataset_train_final))]
    dataset_train_final[, training := 0L]
    
    dataset_train_final[
      (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
      training := 1L
    ]
    
    dataset_train_final[, azar:= NULL] # elimino la columna azar
} else {
    cat("Aplicando CLIENT-WISE undersampling\n")
    dataset_train_final <- clientwise_undersampling(dataset_train_final, PARAM$train_final$undersampling, PARAM$semilla_primigenia)
}

contar_filas(dataset_train_final)


Aplicando NAIVE undersamplingClientes únicos: 179639 
Filas totales: 4060310 


### Target Engineering

In [45]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

# Final Model
Sólo uno. Entreno y entrego

In [ ]:
# utilizo  zLightGBM  la nueva libreria
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")
Sys.time()

In [ ]:
# Ojo: Acá estaba la columna "azar" en el segundo argumento de setdiff()
campos_buenos <- setdiff(
  colnames(dataset_train_final),
  c("clase_ternaria", "clase01", "training")
)


In [ ]:
# dejo los datos en el formato que necesita LightGBM

dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

In [46]:

PARAM$train_final$lgbm$param_completo <-  list(
  boosting= "gbdt",
  objective= "binary",
  metric= "custom",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE,
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_bin= 31L,
  min_data_in_leaf= plocal$min_data_in_leaf,  #este ya es el valor default de LightGBM

  num_iterations= 9999L, # dejo libre la cantidad de arboles, zLightGBM se detiene solo
  num_leaves= 9999L, # dejo libre la cantidad de hojas, zLightGBM sabe cuando no hacer un split
  learning_rate= plocal$learning_rate,  # se lo deja en 1.0 para que si el score esta por debajo de gradient_bound no se lo escale
    
  feature_fraction= 0.50, # un valor equilibrado, habra que probar alternativas ...
    
  canaritos= PARAM$train_final$lgbm$qcanaritos, # fundamental en zLightGBM, aqui esta el control del overfitting
  gradient_bound= plocal$gradient_bound   # default de zLightGBM
)

Sys.time()

[1] "2025-11-14 12:05:45 UTC"

In [ ]:
# Semillerio Final

# No uso APO aquí, sólo semillerío para entregar un ensamble. No importa que ponga
# arriba, APO = 1
PARAM$train_final$APO <- 1
PARAM$train_final$ksemillerio  <- plocal$ksemillerio

# Tampoco necesito cortes
# PARAM$train_final$cortes <- c(8000, 8500, 9000, 9500, 10000, 10500, 11000, 11500, 12000)

In [48]:
# detach("package:lightgbm", unload= TRUE)

In [50]:
if(!require("primes")) install.packages("primes")
require("primes")

Loading required package: primes



In [ ]:
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
# PARAM$train_final$semillas <- sample(primos)[seq( PARAM$train_final$APO*PARAM$train_final$ksemillerio )]
PARAM$train_final$semillas <- sample(primos)[seq(PARAM$train_final$ksemillerio )]
PARAM$train_final$semillas

[1] 103969 851639 751609 405599 720367

In [ ]:
# genero los modelitos
dir.create( "modelitos", showWarnings= FALSE)

param_completo <- copy( PARAM$train_final$lgbm$param_completo)

for( sem in PARAM$train_final$semillas ) {

  arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
  if( !file.exists( arch_modelo ) )
  {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data= dtrain_final,
      param= param_completo
    )

    lgb.save( modelito, filename= arch_modelo)

    # grabo un dataset que tiene el detalle de los arboles de LightGBM
    tb_arboles <- lgb.model.dt.tree(modelito)
    # arch_dataset <- paste0("./modelitos/tb_arboles_", sem, ".txt")
    # fwrite(tb_arboles, file=arch_dataset, sep="\t")

    cat("cantidad arbolitos=", tb_arboles[, max(tree_index)+1],"\n" )
    cat("summary de las hojas de los arboles")
    summary( tb_arboles[, list(hojas=max(leaf_index, na.rm=TRUE)+1), tree_index][,hojas])
    
    # Finalmente lo tiro
    rm(modelito)
    gc()
  }
}

Sys.time()

### Scoring

Se hace el predict() del modelo en los datos del futuro

In [ ]:
dfuture <- dataset[foto_mes %in% PARAM$train_final$future ]

cols0 <- copy(colnames(dfuture))
filas <- nrow(dfuture)

if( PARAM$train_final$lgbm$qcanaritos > 0 ) {
  for( i in seq(PARAM$train_final$lgbm$qcanaritos) ){
    dfuture[, paste0("canarito_",i) := runif( filas) ]
  }

  # las columnas canaritos mandatoriamente van al comienzo del dataset
  cols_canaritos <- copy( setdiff( colnames(dfuture), cols0 ) )
  setcolorder( dfuture, c( cols_canaritos, cols0 ) )
}

In [ ]:
# dataset de future, donde en este caso estoy haciendo testing

mfuture <- data.matrix(dfuture[, campos_buenos, with= FALSE])

# dfuture[, ganancia := ifelse(clase_ternaria=="BAJA+2", 780000, -20000)]

In [ ]:
# mganancias <- matrix( nrow=PARAM$train_final$APO, ncol= length(PARAM$train_final$cortes) )

if( file.exists("prediccion.txt") )
  file.remove("prediccion.txt")

In [ ]:
# aplico el modelo a los datos del future

for( vapo in seq(PARAM$train_final$APO) ) {
  # inicializacion en CERO
  vpred_acum <- rep(0.0, nrow(dfuture))
  qacumulados <- 0

  desde <- 1 + (vapo-1)*PARAM$train_final$ksemillerio
  hasta <- desde + PARAM$train_final$ksemillerio - 1
  semillas <- PARAM$train_final$semillas[desde:hasta]

  for( sem in semillas ) {

    arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
    if( file.exists( arch_modelo ) )
    {
      modelo_final <- lgb.load(arch_modelo) # leo del disco
      # hago el predict() y acumulo
      vpred_acum <- vpred_acum + predict(modelo_final, mfuture)
      qacumulados <- qacumulados + 1
      rm(modelo_final)
      gc()
    }
  }

  if( qacumulados > 0 ) {
    vpred_acum <- vpred_acum / qacumulados  # paso a probabildiad
    # tabla de prediccion, puede ser util para futuros ensembles
    #  ya que le modelo ganador va a ser un ensemble de LightGBMs

    # tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes, ganancia)]
    tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]
    # tb_prediccion[, meta_modelo := vapo]
    tb_prediccion[, prob := vpred_acum ]
    # setorder( tb_prediccion, -prob )
    # tb_prediccion[, gan_acum := cumsum(ganancia)]
    # tb_prediccion[, ganancia := NULL ]

    # acumulo las ganancias
    # for( icor in seq(length(PARAM$train_final$cortes)) ){
      # mganancias[ vapo, icor ] <- tb_prediccion[ PARAM$train_final$cortes[icor], gan_acum ]
    # }

    # grabo las probabilidades del modelo
    fwrite(tb_prediccion,
      file= "prediccion.txt",
      sep= "\t",
      append= TRUE
    )

    # Hay una sola iteración porque APO = 1
    # rm(tb_prediccion)
    # gc()
  }
}

Sys.time()

### Clasificacion

Se tomó la decisión de enviar a los 11000 registros con mayor probabilidad de POS={"BAJA+1","BAJA+"}
<br> esto se determinó en forma artesanal analizando meses anterior
<br> esta es una muy importante decisión 

In [ ]:
# genero archivos con los  "envios" mejores
dir.create("kaggle", showWarnings=FALSE)

# No la borré así que no hace falta cargarla de nuevo
# tb_prediccion <- fread("prediccion.txt")

In [ ]:
# Ya no es de fantasía. ~~genero archivos de fantasia, que NO son el que voy a subir a la Pseudo Competencia Kaggle~~
envios <- 11000
setorder(tb_prediccion, -prob)
tb_prediccion[, Predicted := 0L] # seteo inicial a 0
tb_prediccion[1:envios, Predicted := 1L] # marco los primeros

archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

# grabo el archivo
fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
  file= archivo_kaggle,
  sep= ","
)

# grabo según el formato de entrega definido
# Filtrar solo clientes con Predicted == 1
clientes_seleccionados <- tb_prediccion[Predicted == 1, .(numero_de_cliente)]

# Guardar a CSV con una única columna, sin nombre
file_name <- paste0("./kaggle/sub_", PARAM$experimento, ".csv")
fwrite(clientes_seleccionados,
       file = file_name,
       col.names = FALSE)   # <- sin nombre de columna



# Todo esto lo comento, no hace falta
# for( vapo in seq(PARAM$train_final$APO) ) {
#   if( tb_prediccion[meta_modelo==vapo, .N] > 0 ) {
#     tb_pred <- tb_prediccion[meta_modelo==vapo]
#     setorder( tb_pred, -prob )
#     tb_pred[, Predicted := 0L] # seteo inicial a 0
#     tb_pred[1:envios, Predicted := 1L] # marco los primeros

#     archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", vapo, "_", envios, ".csv")

#     # grabo el archivo
#     fwrite(tb_pred[, list(numero_de_cliente, Predicted)],
#       file= archivo_kaggle,
#       sep= ","
#     )

#     rm( tb_pred )
#     gc()
#   }
# }

Sys.time()